# CommLab Final Project: AQS Implementation with Chained CNOT Encryption

## Import packages

In [1]:
from qiskit import *
import numpy as np
from qiskit.tools.visualization import circuit_drawer
from QKD import QKD

In [2]:
qasm_sim = Aer.get_backend('qasm_simulator')
def get_measurements(qc, num_shots): 
    job = execute(qc, qasm_sim, memory = True, shots = num_shots) 
    result = job.result()
    return result

In [3]:
state_simulator = Aer.get_backend('statevector_simulator')
def state_simulate(circuit): 
    job = execute(circuit, state_simulator)
    result = job.result()
    statevector = result.get_statevector() 
    return statevector

## Initializing phase

In [4]:
def bin_to_int_key(bin_key):
    # transform binary key shared by alice and trent into KAT(integer based key)
    n = len(bin_key)
    int_key = []
    zero = []
    one = []
    for i in range(0,n):
        if not bin_key[i]:
            zero.append(i)
        else:
            one.append(i)
    int_key = zero + one
    return int_key

def generate_message(n):
    message = QuantumRegister(n, name="p")
    message_measure = ClassicalRegister(n, name="p_c")
    qc = QuantumCircuit(message, message_measure)
    for i in range(n):
        random_state = quantum_info.random_statevector(dims=2, seed=None)
        qc.initialize(random_state, message[i])
    return qc

# will be replaced with QKD
def get_shared_key(n):
    # n = len(P)
    r_AT = QKD(n)
    r_BT = QKD(2*n + 1)
    
    K_AT = bin_to_int_key(r_AT)
    K_BT = bin_to_int_key(r_BT)
    
    return K_AT, K_BT, r_BT

def create_bell_states(n):
    A_pos = QuantumRegister(n)
    B_pos = QuantumRegister(n)  
    qc = QuantumCircuit(A_pos, B_pos)
    
    for i in range(n):
        qc.h(A_pos[i])
        qc.cnot(A_pos[i], B_pos[i])
    return qc.to_instruction(label="bell_state")

In [5]:
n = 4
shots = 100

P = QuantumRegister(n, name="p_original")
P_enc = QuantumRegister(n, name="p_enc")
S_A = QuantumRegister(n, name="s_a")
for_M_A = QuantumRegister(n, name="m_a_mea")

M_A_1 = QuantumRegister(n, name="m_a_1")
M_A_2 = QuantumRegister(n, name="m_a_2")

A_bell = QuantumRegister(n, name="alice_bell")
B_bell = QuantumRegister(n, name="bob_bell")

verify = QuantumRegister(1, name="verify")
cswap = QuantumRegister(1, name="cswap")
cswap_2 = QuantumRegister(1, name="cswap_2")
cswap_3 = QuantumRegister(1, name="cswap_3")
cswap_4 = QuantumRegister(1, name="cswap_4")
cswap_5 = QuantumRegister(1, name="cswap_5")

verify_measure = ClassicalRegister(1, name="verify_measure")
cswap_measure = ClassicalRegister(1, name="cswap_measure")
cswap_measure_2 = ClassicalRegister(1, name="cswap_measure_2")
cswap_measure_3 = ClassicalRegister(1, name="cswap_measure_3")
cswap_measure_4 = ClassicalRegister(1, name="cswap_measure_4")
cswap_measure_5 = ClassicalRegister(1, name="cswap_measure_5")

qc = QuantumCircuit(P, P_enc, S_A, M_A_1, M_A_2, 
                    A_bell, B_bell, for_M_A, verify, verify_measure,
                    cswap, cswap_2, cswap_3, cswap_4, cswap_5,
                    cswap_measure, cswap_measure_2, cswap_measure_3, cswap_measure_4, cswap_measure_5)

get_M_A = []

for m in range(2*n):
    get_M_A.append(ClassicalRegister(1, 'mac_' + str(m)))
    qc.add_register(get_M_A[m])

measure_M_A_1 = []
measure_M_A_2 = []
for m in range(n):
    measure_M_A_1.append(ClassicalRegister(1, 'm_a_1_measure'+str(m)))
    measure_M_A_2.append(ClassicalRegister(1, 'm_a_2_measure'+str(m)))
    qc.add_register(measure_M_A_1[m])
    qc.add_register(measure_M_A_2[m])

# Step 1 : 3 copies of the message
for i in range(n):
    random_state = quantum_info.random_statevector(dims=2, seed=None)
    qc.initialize(random_state, P_enc[i])
    qc.initialize(random_state, S_A[i])
    qc.initialize(random_state, for_M_A[i])
    qc.initialize(random_state, P[i])

# keys generation
K_AT, K_BT, r_BT = get_shared_key(n)

# Step 2 : generate bell states between Alice, Bob
AB_bell = []
for i in range(n):
    AB_bell.append(A_bell[i])
for i in range(n):
    AB_bell.append(B_bell[i])
qc.append(create_bell_states(n), AB_bell)

/Users/macbookpro/opt/anaconda3/envs/IBMQ/lib/python3.9/site-packages/qiskit/utils/deprecation.py:62: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  return func(*args, **kwargs)


In [6]:
print(K_AT)
print(K_BT)

[1, 0, 2, 3]
[1, 2, 3, 4, 6, 7, 0, 5, 8]


## Signing phase

In [7]:
def chained_cnot_enc(n, K):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    for i in range(n):
        if i != K[i]:
            qc.cnot(i, K[i])
    return qc.to_instruction(label="chained_cnot_enc")

def chained_cnot_dec(n, K):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    print()
    for i in reversed(range(0,n)):
        if i != K[i]:
            qc.cnot(i, K[i])
    return qc.to_instruction(label="chained_cnot_dec")

def bell_measurement(n):
    q1 = QuantumRegister(n)
    q2 = QuantumRegister(n)
    qc = QuantumCircuit(q1, q2)
    for i in range(n):
        qc.cnot(q1[i], q2[i])
        qc.h(q1[i])
    return qc.to_instruction(label="bell_measurement")

In [8]:
# Step 1 : 1st copy of P --> P_enc
r_R = np.random.randint(0, 2, n)
K_R = bin_to_int_key(r_R)

qc.append(chained_cnot_enc(n, K_R), P_enc)

# Step 2 : 2nd copy of P --> P_enc --> S_A
qc.append(chained_cnot_enc(n, K_R), S_A)
qc.append(chained_cnot_enc(n, K_AT), S_A)

# Step 3 : generate one type of bell state (to transform into other bell states later)
bell_m = []

for i in range(n):
    bell_m.append(M_A_1[i])
for i in range(n):
    bell_m.append(M_A_2[i])
qc.append(create_bell_states(n), bell_m) 

# 3rd copy of P -> P_enc
qc.append(chained_cnot_enc(n, K_R), for_M_A) 

bell_m = []
# bell_m is P_enc and A_bell concatenated

for i in range(n):
    bell_m.append(A_bell[i]) # A's bell state
for i in range(n): 
    bell_m.append(for_M_A[i]) # P_enc

# do bell measurement on A's bell and P_enc
qc.append(bell_measurement(n), bell_m) 

In [9]:
for i in range(n):
    qc.measure(A_bell[i], get_M_A[2*i])
    qc.measure(for_M_A[i], get_M_A[2*i+1])
    qc.z(M_A_1[i]).c_if(get_M_A[2*i], 1)
    qc.x(M_A_2[i]).c_if(get_M_A[2*i+1], 1)

## Verification Phase

In [10]:
def cswap_comparison(n):
    # q1, q2 : qubits to compare
    # for_measure : the qubits to measure
    q1 = QuantumRegister(n)
    q2 = QuantumRegister(n)
    for_measure = QuantumRegister(1)
    qc = QuantumCircuit(q1, q2, for_measure)
    qc.h(for_measure[0])
    for i in range(n):
        qc.cswap(for_measure[0], q1[i], q2[i])
    qc.h(for_measure[0])
    return qc.to_instruction(label="swap test")
    # cswap measurement = 0 --> same

In [11]:
# step 1 : (P_enc, S_A) --> (P_enc, S_A)'
P_enc_S_A = [*P_enc, *S_A]
for i in range(2*n):
    if r_BT[i] == 1 :
        qc.x(P_enc_S_A[i])

# step 2 : YB = encode K_BT(P_enc, S_A)'
K_BT_2n = K_BT[:]
K_BT_2n.remove(2*n)
qc.append(chained_cnot_enc(2*n, K_BT_2n), [*P_enc, *S_A])

# decrypt : YB --> (P_enc, S_A)'
qc.append(chained_cnot_dec(2*n, K_BT_2n), [*P_enc, *S_A])

# decrypt : (P_enc, S_A)' --> (P_enc, S_A)
for i in range(2*n):
    if r_BT[i] == 1 :
        qc.x(P_enc_S_A[i])

qc.append(chained_cnot_enc(n, K_AT), P_enc)

In [12]:
qc.draw(fold=-1)

┌─────────────────────────────────────────────────┐                                                                                                                                                                                    
     p_original_0: ┤ Initialize(-0.53325+0.052908j,0.14724+0.83136j) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   ├─────────────────────────────────────────────────┤                                                                                                                                                                                    
     p_original_1: ┤ Initialize(-0.21416-0.37918j,-0.55757-0.70674j) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   ├─────────────────────────────────────────────────┤                                                                                                                                                                                    
     p_original_2: ┤ Initialize(0.64669-0.74551j,-0.16003-0.020252j) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   └─┬──────────────────────────────────────────────┬┘                                                                                                                                                                                    
     p_original_3: ──┤ Initialize(0.93757-0.1344j,0.29372+0.12892j) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                   ┌─┴──────────────────────────────────────────────┴┐┌───────────────────┐        ┌───┐             ┌───────────────────┐┌───────────────────┐┌───┐┌───────────────────┐                                                                 
          p_enc_0: ┤ Initialize(-0.53325+0.052908j,0.14724+0.83136j) ├┤0                  ├────────┤ X ├─────────────┤0                  ├┤0                  ├┤ X ├┤0                  ├─────────────────────────────────────────────────────────────────
                   ├─────────────────────────────────────────────────┤│                   │        └───┘             │                   ││                   │└───┘│                   │                                                                 
          p_enc_1: ┤ Initialize(-0.21416-0.37918j,-0.55757-0.70674j) ├┤1                  ├──────────────────────────┤1                  ├┤1                  ├─────┤1                  ├─────────────────────────────────────────────────────────────────
                   ├─────────────────────────────────────────────────┤│  chained_cnot_enc │                          │                   ││                   │     │  chained_cnot_enc │                                                                 
          p_enc_2: ┤ Initialize(0.64669-0.74551j,-0.16003-0.020252j) ├┤2                  ├──────────────────────────┤2                  ├┤2                  ├─────┤2                  ├─────────────────────────────────────────────────────────────────
                   └─┬──────────────────────────────────────────────┬┘│                   │                          │                   ││                   │     │                   │                                                                 
          p_enc_3: ──┤ Initialize(0.93757-0.1344j,0.29372+0.12892j) ├─┤3                  ├──────────────────────────┤3                  ├┤3                  ├─────┤3                  ├─────────────────────────────────────────────────────────────────
   

In [13]:
# swap test
qc.append(cswap_comparison(n), [*P_enc, *S_A, *cswap])
qc.measure(cswap, cswap_measure)

result = get_measurements(qc, shots)
counts = result.get_counts()
print(counts)

qc.append(chained_cnot_dec(n, K_AT), P_enc)

{'0 0 0 0 0 0 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 0 1 0 0 1 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 0': 1, '0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0': 1,

In [14]:
# check if verifiable 
identical = True
for key in counts:
    if key[-3] != '0':
        identical = False
print(identical)

True


In [15]:
# Step 3 : Trent sends all to Bob
if identical :
    qc.x(verify)
qc.append(chained_cnot_enc(2*n+1, K_BT), [*P_enc, *S_A, *verify])

In [16]:
# Step 4 : Bob decrypts YTB and measures verify bit
qc.append(chained_cnot_dec(2*n+1, K_BT), [*P_enc, *S_A, *verify])
qc.measure(verify, verify_measure)

result = get_measurements(qc, shots)
counts = result.get_counts()
print(counts)


{'0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 1': 1, '0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1': 1

In [17]:
verified = True
for key in counts : 
    if key[-1] != '1':
        verified = False
print(verified)

True


In [18]:
identical = True
for key in counts:
    if key[-5] != "0":
        identical = False
print(identical)

True


In [19]:
if not identical: 
    print("Abort")
else : 
    print("Alice publishes KR")
    qc.append(chained_cnot_dec(n, K_R), P_enc) # Bob obtains P_original
    
    # additional checking for debug purposes (P_DECODED = P_ORIGINAL)
    # below swap test is not in the protocol : 
    # qc.append(cswap_comparison(n), [*P, *P_enc, *cswap_3])
    # qc.measure(cswap_3, cswap_measure_3)
    
    # result = get_measurements(qc, shots)
    # counts_s = result.get_counts()

Alice publishes KR



In [20]:
# identical = True
# for key in counts:
#     if key[-7] != "0":
#         identical = False
# print(identical)

## Dispute Resolve

In [21]:
# BOB : P, K_R, S_A
# TRENT calculates S_T : 
qc.append(chained_cnot_enc(n, K_R), P_enc)
qc.append(chained_cnot_enc(n, K_AT), P_enc)
qc.append(cswap_comparison(n), [*P_enc, *S_A, *cswap_4])
qc.measure(cswap_4, cswap_measure_4)

result = get_measurements(qc, shots)
counts_s = result.get_counts()

# restore Bob's P original(stored in P_enc)
qc.append(chained_cnot_dec(n, K_AT), P_enc)
qc.append(chained_cnot_dec(n, K_R), P_enc)

In [22]:
identical = True
for key in counts:
    if key[-9] != "0":
        identical = False
if identical :
    print("Message, signature pair valid")
else :
    print("Message, signature pair invalid")

Message, signature pair valid


## Bob's Forgery : Known Message Attack

In [23]:
# Bob now has a valid signature, message pair (S, P) 
# step 1 : generate random string r and add same pauli gates on S, P 
rdstr = []
r = np.random.randint(4, size=n)
print(r)

[1 2 0 3]


In [24]:
for i in range(n):
    if r[i] % 4 == 0:
        qc.x(P_enc[i])
        qc.x(S_A[i])
    elif r[i] % 4 == 1:
        qc.z(P_enc[i])
        qc.z(S_A[i])
    elif r[i] % 4 == 2:
        qc.x(P_enc[i])
        qc.z(P_enc[i])
        qc.x(S_A[i])
        qc.z(S_A[i])

In [25]:
# Trent : dispute resolve
qc.append(chained_cnot_enc(n, K_R), P_enc)
qc.append(chained_cnot_enc(n, K_AT), P_enc)
qc.append(cswap_comparison(n), [*P_enc, *S_A, *cswap_5])
qc.measure(cswap_5, cswap_measure_5)

result = get_measurements(qc, shots)
counts_s = result.get_counts()

# restore Bob's P original(stored in P_enc)
qc.append(chained_cnot_dec(n, K_AT), P_enc)
qc.append(chained_cnot_dec(n, K_R), P_enc)

In [26]:
identical = True
c = 0
for key in counts_s:
    if key[-11] != "0":
        identical = False
        c += 1
if identical :
    print("Bob's forgery succeeded")
else : 
    print("Bob's forgery failed")
    print("Bob succeeds with probability :")
    print(c /shots)

Bob's forgery failed
Bob succeeds with probability :
0.45
